In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recsys').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RankingEvaluator

In [3]:
df = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [5]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train_data, test_data = df.randomSplit([.8, .2])

In [8]:
als = ALS(maxIter=10, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [9]:
model = als.fit(train_data)

In [10]:
prediction = model.transform(test_data)

In [11]:
prediction.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   1.0|    26| 2.5159435|
|      2|   2.0|     1| 1.6715983|
|      4|   2.0|     1| 1.5265448|
|      0|   1.0|     6|-0.1965212|
|      2|   1.0|    16| 2.1065145|
|      1|   1.0|     5| 1.2873783|
|      0|   1.0|    19| 1.6886952|
|      0|   1.0|    15| 2.4492357|
|      1|   4.0|    15| 0.8127151|
|      2|   1.0|    15| 1.7605423|
|      2|   3.0|     9| 1.5802273|
|      3|   1.0|     9|  2.481995|
|      4|   2.0|     8| 0.6139077|
|      5|   1.0|     8| 1.3301669|
|      3|   1.0|     7| 1.3409852|
|      1|   3.0|    25|0.37358713|
|      4|   1.0|    24| 2.8245494|
|      4|   1.0|    29| 1.6529197|
|      3|   1.0|    21| 2.0982878|
|      1|   1.0|    14| 1.2720944|
+-------+------+------+----------+
only showing top 20 rows



In [12]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId', 'userId'])

In [13]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     16|    11|
|     35|    11|
|     43|    11|
|     69|    11|
|     75|    11|
|     76|    11|
|     97|    11|
+-------+------+



In [14]:
recommendation = model.transform(single_user)
recommendation.orderBy('prediction', ascending= False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     35|    11| 2.5414002|
|     76|    11|  2.388986|
|     16|    11| 1.9703206|
|     69|    11| 1.7795041|
|     75|    11| 1.6127572|
|     97|    11| 1.4134268|
|     43|    11| 1.3339801|
|     11|    11|0.77664554|
+-------+------+----------+

